In [0]:
!pip install -q keras

In [2]:
import numpy as np
from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import one_hot
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding

Using TensorFlow backend.


In [4]:
!pip install --upgrade gensim

    100% |████████████████████████████████| 23.6MB 2.0MB/s 
    100% |████████████████████████████████| 1.4MB 15.1MB/s 
    100% |████████████████████████████████| 133kB 26.9MB/s 
    100% |████████████████████████████████| 4.8MB 6.2MB/s 
    100% |████████████████████████████████| 61kB 22.6MB/s 
    100% |████████████████████████████████| 552kB 24.4MB/s 
  Running setup.py bdist_wheel for smart-open ... - \ done
  Stored in directory: /root/.cache/pip/wheels/23/00/44/e5b939f7a80c04e32297dbd6d96fa3065af89ecf57e2b5f89f
  Running setup.py bdist_wheel for bz2file ... - done
  Stored in directory: /root/.cache/pip/wheels/81/75/d6/e1317bf09bf1af5a30befc2a007869fa6e1f516b8f7c591cb9
Successfully built smart-open bz2file


In [47]:
# define documents
docs = ['Well done!',
        'Good work',
        'Great effort',
        'nice work',
        'Excellent!',
        'Weak',
        'Poor effort!',
        'not good',
        'poor work',
        'Could have done better.', 'Very Good', 'can be better', 'very poor']

# define class labels
labels = array([1,1,1,1,1,0,0,0,0,0,1,0,0])


# integer encode the documents
vocab_size = 50
encoded_docs = [one_hot(d, vocab_size) for d in docs]
print(encoded_docs, '\n')

# pad documents to a max length of 4 words
max_length = 4
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs, '\n')

train_doc, train_label = padded_docs[:-3], labels[:-3]

# define the model
model = Sequential()
model.add(Embedding(vocab_size, 8, input_length=max_length))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# summarize the model
print(model.summary(), '\n')
# fit the model
model.fit(train_doc, train_label, epochs=50, verbose=0)
# evaluate the model

loss, accuracy = model.evaluate(train_doc, train_label, verbose=0)
print('Accuracy: %f' % (accuracy*100), '\n')

doc_test = padded_docs[-3:]
print (docs[-3:])
ynew = model.predict_classes(doc_test)
pnew = model.predict_proba(doc_test)
for i in range(len(doc_test)):
    print("Y=%s, Predicted=%s" % (ynew[i], pnew[i]))

[[39, 29], [5, 8], [4, 1], [24, 8], [35], [31], [32, 1], [25, 5], [32, 8], [8, 39, 29, 36], [34, 5], [16, 16, 36], [34, 32]] 

[[39 29  0  0]
 [ 5  8  0  0]
 [ 4  1  0  0]
 [24  8  0  0]
 [35  0  0  0]
 [31  0  0  0]
 [32  1  0  0]
 [25  5  0  0]
 [32  8  0  0]
 [ 8 39 29 36]
 [34  5  0  0]
 [16 16 36  0]
 [34 32  0  0]] 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (None, 4, 8)              400       
_________________________________________________________________
flatten_7 (Flatten)          (None, 32)                0         
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 33        
Total params: 433
Trainable params: 433
Non-trainable params: 0
_________________________________________________________________
None 

Accuracy: 80.000001 

['Very Good', 'can be better', 'very poor']
Y=[0], Pre

# Word2Vec


Using Pre-Trained GloVe Embedding

In [8]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip  

--2018-11-05 22:47:53--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2018-11-05 22:47:54--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  23.7MB/s    in 29s     

2018-11-05 22:48:22 (28.8 MB/s) - ‘glove.6B.zip’ saved [862182613/862182613]

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [9]:
!ls
!pwd

glove.6B.100d.txt  glove.6B.300d.txt  glove.6B.zip
glove.6B.200d.txt  glove.6B.50d.txt   sample_data
/content


In [10]:
# define documents
docs = ['Well done!',
        'Good work',
        'Great effort',
        'nice work',
        'Excellent!',
        'Weak',
        'Poor effort!',
        'not good',
        'poor work',
        'Could have done better.', 'Good', 'can not be better', 'poor']

# define class labels
labels = array([1,1,1,1,1,0,0,0,0,0,1,0,0])
# prepare tokenizer
t = Tokenizer()
t.fit_on_texts(docs)
vocab_size = len(t.word_index) + 1
# integer encode the documents
encoded_docs = t.texts_to_sequences(docs)
print(encoded_docs)
# pad documents to a max length of 4 words
max_length = 4
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)

train_doc, train_label = padded_docs[:-3], labels[:-3]



# load the whole embedding into memory
embeddings_index = dict()
f = open('glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))
# create a weight matrix for words in training docs
embedding_matrix = zeros((vocab_size, 100))
for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
        
# define model
model = Sequential()
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=4, trainable=False)
model.add(e)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# summarize the model
print(model.summary())
# fit the model
model.fit(train_doc, train_label, epochs=50, verbose=0)
# evaluate the model
loss, accuracy = model.evaluate(train_doc, train_label, verbose=0)
print('Accuracy: %f' % (accuracy*100))

[[8, 4], [1, 2], [9, 5], [10, 2], [11], [12], [3, 5], [6, 1], [3, 2], [13, 14, 4, 7], [1], [15, 6, 16, 7], [3]]
[[ 8  4  0  0]
 [ 1  2  0  0]
 [ 9  5  0  0]
 [10  2  0  0]
 [11  0  0  0]
 [12  0  0  0]
 [ 3  5  0  0]
 [ 6  1  0  0]
 [ 3  2  0  0]
 [13 14  4  7]
 [ 1  0  0  0]
 [15  6 16  7]
 [ 3  0  0  0]]
Loaded 400000 word vectors.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 4, 100)            1700      
_________________________________________________________________
flatten_1 (Flatten)          (None, 400)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 401       
Total params: 2,101
Trainable params: 401
Non-trainable params: 1,700
_________________________________________________________________
None
Accuracy: 100.000000


In [11]:
doc_test = padded_docs[-3:]
print (docs[-3:])
ynew = model.predict_classes(doc_test)
pnew = model.predict_proba(doc_test)
for i in range(len(doc_test)):
    print("Y=%s, Predicted=%s" % (ynew[i], pnew[i]))

['Good', 'can not be better', 'poor']
Y=[1], Predicted=[0.64661646]
Y=[0], Predicted=[0.0947505]
Y=[0], Predicted=[0.37935227]


# Sequence Classification with LSTM for IMDB

Develope an LSTM model for sequence classification problem using the Keras IMDB dataset. 
I also use Convolutional Neural Networks to capature the spatial relationships (5 or several words together)

See the networks developed, the first layer is an embedding layer. It turns positive integers into dense vectors (embedding vectors) of fixed size , which can then be used by convolution layer. Eg. [[4], [20]] -> [[0.25, 0.1], [0.6, -0.2]] The input of embedding layer can only be positive integers, which can be provided by Tokenizer (see the "moive_review" example below). The Embedding layer is initialized with random weights, and will be trained from all the words in the training dataset.

The Conv1D layer is an one-dimension convolution layer, which can capature the sequence of words (n-grams). 
![](https://raw.githubusercontent.com/eisenjulian/nlp_estimator_tutorial/master/conv.png)

      Soure: Learning to Rank Short Text Pairs with Convolutional Deep Neural Networks by Severyn et al. [2015]

The general idea of LSTM is:

![](https://cdn-images-1.medium.com/max/800/1*laH0_xXEkFE0lKJu54gkFQ.png)

         source Understanding LSTM and its diagrams by Shi Yan 

Please see the "Understanding LSTM and its diagrams" by Shi Yan (https://medium.com/mlreview/understanding-lstm-and-its-diagrams-37e2f46f1714) and Adrew Ng's course for details about LSTM or GRU (https://www.coursera.org/lecture/nlp-sequence-models/long-short-term-memory-lstm-KXoay).


In [14]:
from keras.datasets import imdb
from keras.preprocessing import sequence
from keras.layers import LSTM, Dropout
from keras.layers.convolutional import Conv1D, MaxPooling1D

# fix random seed for reproducibility
np.random.seed(7)
# load the dataset but only keep the top n words, zero the rest
top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)

#X_train, y_train = X_train[:int(len(X_train)/2)], y_train[:int(len(y_train)/2)]
# truncate and pad input sequences
max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)
# create the model
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
# num of parameter: num_filters * (kernel_size * embedding_vecor_length) + num_filters = 32*(3*32)+32=3104
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(100))
# num of parameter: 4(n*m + n^2 +n)= 4*(100*32 + 100.^2 + 100) = 53200
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, epochs=3, batch_size=64)


# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 500, 32)           3104      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 250, 32)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 101       
Total params: 216,405
Trainable params: 216,405
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/3
25000/25000 [==============================] - 285s 11ms/step - loss: 0.4363 - acc: 0.7847
Epoch 2/3
25000/25000 [=======